In [1]:
import sys
import os

# Add parent directory to the Python path
sys.path.append(os.path.abspath('..'))
from utils import PapuGaPT2Handler

# Initialize our model handler
handler = PapuGaPT2Handler()

# System & conversation setup
system_prompt = (
    "You are an AI assistant speaking with a human.\n"
    "AI: Hello! How can I help you today?\n"
)

# We'll store all messages (system, AI, and human) in a list
conversation_history = [system_prompt]

/pio/scratch/1/i317214/miniconda/envs/hallucination_detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Loading model: flax-community/papuGaPT2


In [2]:
def summarize_messages(messages, handler):
    """
    Toy function that tries to summarize the last messages using the model itself.
    This is *very* simplistic and might not produce a high-quality summary.
    """
    # Simply join the last 6 messages
    text_to_summarize = "\n".join(messages[-6:])
    # Ask the model to "summarize" them:
    prompt = f"Proszę streść następującą konwersację:\n{text_to_summarize}\n\nPodsumowanie:"
    
    # We'll generate a single response
    summary_candidates = handler.generate_text(prompt, max_length=60, num_return_sequences=1)
    summary = summary_candidates[0]
    
    # We only want the "summary" part after "Podsumowanie:" to keep it short
    if "Podsumowanie:" in summary:
        summary = summary.split("Podsumowanie:")[-1].strip()
    
    return "Oto podsumowanie poprzedniej konwersacji:\n" + summary

In [3]:
def chat_with_model(user_input, conversation_history, handler):
    """
    1. Takes user input and the current conversation history.
    2. Generates multiple candidate replies (3 by default).
    3. Picks the best reply based on NLL (lowest is best).
    4. Adds best reply to conversation history and returns it.
    5. If messages exceed a certain threshold, summarize last 6 messages.
    """
    # 1) Add user message to the conversation
    conversation_history.append(f"Human: {user_input}")
    
    # 2) Construct the prompt from conversation history
    # We'll just join everything so far
    prompt = "\n".join(conversation_history) + "\nAI:"
    
    # 3) Generate three responses
    candidates = handler.generate_text(
        prompt=prompt,
        max_length=60,
        num_return_sequences=3,
        temperature=1.0,
        top_k=50,
        top_p=0.95
    )
    
    # 4) Pick the best reply by measuring NLL
    best_reply = None
    best_score = float('inf')
    for cand in candidates:
        # We only want the part that comes *after* the last 'AI:' if the model copies the prompt
        reply_candidate = cand.split("AI:")[-1].strip()
        
        # Evaluate NLL
        score = handler.sentence_probability(reply_candidate)
        if score < best_score:
            best_score = score
            best_reply = reply_candidate

    # 5) Add chosen AI reply to conversation history
    conversation_history.append(f"AI: {best_reply}")

    # 6) If messages are too long, summarize the last 6
    #    Here we treat each 'Human: ...' or 'AI: ...' as a single message
    if len(conversation_history) > 12:
        summary = summarize_messages(conversation_history[-6:], handler)
        conversation_history[:] = [summary]  # Reset the history with the summary as new 'system' context
    
    return conversation_history


In [5]:
# We can now do an interactive loop in a notebook cell:
while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ["quit", "exit"]:
        print("Exiting chat. Bye!")
        break
    
    conversation_history = chat_with_model(user_input, conversation_history, handler)
    # Print the latest AI reply
    print(conversation_history[-1])  # The last line should be "AI: <something>"


AI: Zbliżają się wakacje, i


ValueError: Input length of input_ids is 73, but `max_length` is set to 60. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.